# Short Integer Solutions (SIS)

SIS is one of the fundamental hard problems in lattice crypto. Introduced by Ajtai in 1996, it forms the basis for collision-resistant hash functions and signature schemes.

The basic idea: given a random matrix A, find a short nonzero vector z such that Az = 0 (mod q). Sounds simple, but finding short solutions is believed to be hard!

Credits: Alfred Menezes - [Lattice-Based Cryptography Course](https://www.youtube.com/playlist?list=PLA1qgQLL41STNFDvPJRqrHtuz0PIEJ4a8)

## What is SIS?

**SIS(n, m, q, B)**: Given a random matrix A ∈ Z_q^(n×m), find a nonzero vector z ∈ Z^m such that:
- Az = 0 (mod q)
- z is short: each entry is in [-B, B]
- B << q/2 (the bound B is much smaller than q)

Parameters:
- n: number of rows (security parameter, typically 256-1024)
- m: number of columns (should be > n)
- q: modulus (prime, typically 3329 or 8380417)
- B: bound on solution coefficients (small, like 1-3)

The key insight: solutions exist (by pigeonhole principle), but finding short ones is hard!

In [ ]:
import numpy as np

# Tiny SIS example: n=2, m=3, q=11, B=2
n, m, q, B = 2, 3, 11, 2

# Random matrix A
A = np.array([
    [3, 5, 7],
    [2, 8, 4]
])

print("SIS Example")
print("=" * 50)
print(f"Parameters: n={n}, m={m}, q={q}, B={B}")
print(f"\nMatrix A:")
print(A)

# Try a solution z = [1, -1, 1]^T
z = np.array([1, -1, 1])

result = (A @ z) % q
print(f"\nTrying solution z = {z}")
print(f"Az mod {q} = {result}")

# Check if it's a valid SIS solution
is_zero = np.all(result == 0)
is_short = np.all(np.abs(z) <= B)
is_nonzero = np.any(z != 0)

print(f"\nValid SIS solution?")
print(f"  Az = 0 (mod {q}): {is_zero}")
print(f"  z ∈ [-{B},{B}]^m: {is_short}")
print(f"  z ≠ 0: {is_nonzero}")
print(f"  Result: {'✓ YES' if (is_zero and is_short and is_nonzero) else '✗ NO'}")

## When Do Solutions Exist?

For SIS solutions to exist with high probability, we need the pigeonhole principle:
- Number of possible short vectors: (B+1)^m (roughly)
- Number of possible outputs: q^n
- If (B+1)^m > q^n, then by pigeonhole principle, two different z vectors must map to the same output

This means we need: m > (n log q) / log(B+1)

Also, we want n < m (otherwise Az=0 typically only has the trivial solution z=0).

In [ ]:
# Check if parameters guarantee solution existence
import math

def check_sis_parameters(n, m, q, B):
    """Check if SIS parameters guarantee solution existence"""
    short_vectors = (2*B + 1) ** m
    outputs = q ** n
    
    min_m = (n * math.log(q)) / math.log(B + 1)
    
    print(f"SIS({n}, {m}, {q}, {B})")
    print("-" * 50)
    print(f"Possible short vectors: (2B+1)^m = {short_vectors:.2e}")
    print(f"Possible outputs: q^n = {outputs:.2e}")
    print(f"Ratio: {short_vectors / outputs:.2f}")
    
    if short_vectors > outputs:
        print("✓ By pigeonhole principle, solutions exist!")
    else:
        print("✗ Not enough short vectors, solutions may not exist")
    
    print(f"\nMinimum m needed: {min_m:.1f}")
    print(f"Actual m: {m}")
    print(f"n < m? {n < m}")
    print()

# Example parameters
check_sis_parameters(3, 5, 13, 3)
check_sis_parameters(256, 512, 3329, 2)
check_sis_parameters(2, 2, 11, 2)  # Bad parameters

## Solving SIS: The Hard Way

To find an SIS solution, we need to:
1. Find the null space of A (mod q) using Gaussian elimination
2. Express the general solution in terms of free variables
3. Search for values of free variables that give short solutions

Let's work through the classic example: n=3, m=5, q=13, B=3

Matrix A:
```
[1   0   7  12   4]
[2  11   3   6  12]
[9   8  10   5   1]
```

In [ ]:
# Gaussian elimination mod 13 to find null space
q = 13
A = np.array([
    [1, 0, 7, 12, 4],
    [2, 11, 3, 6, 12],
    [9, 8, 10, 5, 1]
])

print("Original matrix A:")
print(A)
print()

# Work with a copy
R = A.copy()

# Helper to find modular inverse
def modinv(a, m):
    """Find modular inverse of a mod m"""
    for i in range(1, m):
        if (a * i) % m == 1:
            return i
    return None

print("Gaussian elimination mod 13:")
print("=" * 50)

# Row 2 -= 2*Row 1
R[1] = (R[1] - 2 * R[0]) % q
print("\nAfter R2 <- R2 - 2*R1:")
print(R)

# Row 3 -= 9*Row 1
R[2] = (R[2] - 9 * R[0]) % q
print("\nAfter R3 <- R3 - 9*R1:")
print(R)

# Row 2 *= inverse of 11 (which is 6 mod 13)
inv_11 = modinv(11, 13)
R[1] = (inv_11 * R[1]) % q
print(f"\nAfter R2 <- {inv_11}*R2 (normalize):")
print(R)

# Row 3 -= 8*Row 2
R[2] = (R[2] - 8 * R[1]) % q
print("\nAfter R3 <- R3 - 8*R2:")
print(R)

# Row 3 *= inverse of 7 (which is 2 mod 13)
inv_7 = modinv(7, 13)
R[2] = (inv_7 * R[2]) % q
print(f"\nAfter R3 <- {inv_7}*R3 (normalize):")
print(R)

# Row 1 -= 7*Row 3
R[0] = (R[0] - 7 * R[2]) % q
print("\nAfter R1 <- R1 - 7*R3:")
print(R)

# Row 2 -= 12*Row 3
R[1] = (R[1] - 12 * R[2]) % q
print("\nAfter R2 <- R2 - 12*R3:")
print(R)

print("\n" + "=" * 50)
print("Reduced row echelon form:")
print(R)
print("\nGeneral solution (z4, z5 are free variables):")
print("z1 = 8*z4 + 3*z5")
print("z2 = 3*z4 + 1*z5")
print("z3 = 12*z4 + 12*z5")
print("z4, z5 ∈ Z_13")

## Finding Short Solutions

Now we have the general solution, but we need solutions where all entries are in [-3, 3].

We'll enumerate all possible (z4, z5) pairs and check which give short solutions.

In [ ]:
# Search for short solutions
B = 3
q = 13

def to_centered(x, q):
    """Convert mod q element to [-q//2, q//2] range"""
    return x if x <= q//2 else x - q

solutions = []

print("Searching for solutions in [-3, 3]^5...")
print("=" * 70)

for z4 in range(q):
    for z5 in range(q):
        # Compute z using the parametric solution
        z1 = (8 * z4 + 3 * z5) % q
        z2 = (3 * z4 + 1 * z5) % q
        z3 = (12 * z4 + 12 * z5) % q
        
        # Convert to centered representation
        z = np.array([to_centered(z1, q), 
                      to_centered(z2, q), 
                      to_centered(z3, q),
                      to_centered(z4, q), 
                      to_centered(z5, q)])
        
        # Check if short and nonzero
        if np.all(np.abs(z) <= B) and np.any(z != 0):
            solutions.append(tuple(z))

# Remove duplicates
solutions = sorted(set(solutions))

print(f"\nFound {len(solutions)} SIS solutions:\n")
for i, sol in enumerate(solutions, 1):
    # Verify it's actually a solution
    z_arr = np.array(sol)
    result = (A @ z_arr) % q
    check = "✓" if np.all(result == 0) else "✗"
    print(f"{i}. z = {sol}  {check}")

print(f"\nExpected 6 solutions (3 pairs ±z):")
print("  ±(3, 1, -1, 0, 1)")
print("  ±(1, 0, -2, -1, 3)")
print("  ±(2, 1, 1, 1, -2)")

## SIS for Collision-Resistant Hashing

One of the main applications of SIS is building collision-resistant hash functions. The idea:

**Hash function**: H_A(z) = Az (mod q)
- Input: z ∈ {0,1}^m (m-bit string)
- Output: Az ∈ Z_q^n (n elements mod q)

**Compression**: Since m > n log q, we have 2^m > q^n, so this compresses the input.

**Collision resistance**: If you can find z1 ≠ z2 with H_A(z1) = H_A(z2), then:
- Az1 = Az2 (mod q)
- So A(z1 - z2) = 0 (mod q)
- And z = z1 - z2 is a short SIS solution (entries in {-1, 0, 1} since z1, z2 ∈ {0,1}^m)

Thus, breaking the hash function = solving SIS!

In [ ]:
# SIS-based hash function example
n, m, q = 4, 8, 257  # Small parameters

# Random public matrix A
np.random.seed(42)
A = np.random.randint(0, q, size=(n, m))

def hash_sis(message_bits):
    """Hash a binary message using SIS"""
    if len(message_bits) != m:
        raise ValueError(f"Message must be {m} bits")
    z = np.array(message_bits, dtype=int)
    return (A @ z) % q

print("SIS-based Hash Function")
print("=" * 70)
print(f"Parameters: n={n}, m={m}, q={q}")
print(f"Input: {m}-bit strings")
print(f"Output: {n} elements mod {q}")
print(f"Compression ratio: {m} bits -> {n * np.log2(q):.1f} bits\n")

# Hash some messages
msg1 = [1, 0, 1, 1, 0, 0, 1, 0]
msg2 = [0, 1, 0, 1, 1, 0, 0, 1]

h1 = hash_sis(msg1)
h2 = hash_sis(msg2)

print(f"Message 1: {msg1}")
print(f"  Hash: {h1}")
print(f"\nMessage 2: {msg2}")
print(f"  Hash: {h2}")
print(f"\nCollision? {np.array_equal(h1, h2)}")

# Finding a collision = finding z1-z2 that solves SIS
print("\n" + "=" * 70)
print("To find a collision:")
print("  Need z1, z2 ∈ {0,1}^m with Az1 = Az2 (mod q)")
print("  Equivalent to finding z = z1-z2 where Az = 0 (mod q)")
print("  and z ∈ {-1,0,1}^m")
print("  This is an SIS problem with B=1!")

## Inhomogeneous SIS (ISIS)

Sometimes we need to solve Az = b (mod q) instead of Az = 0. This is called ISIS:

**ISIS(n, m, q, B)**: Given A ∈ Z_q^(n×m) and b ∈ Z_q^n, find z ∈ Z^m such that:
- Az = b (mod q)
- z ∈ [-B, B]^m

ISIS is useful for signature schemes where you need to "sign" a message represented by b.

The good news: SIS and ISIS are equivalent! You can reduce one to the other efficiently.

In [ ]:
# ISIS example: Az = b (mod q)
q = 17
A = np.array([
    [3, 5, 7, 2],
    [8, 1, 4, 9]
])
b = np.array([10, 5])

print("ISIS Example")
print("=" * 50)
print(f"Matrix A:")
print(A)
print(f"\nTarget b: {b}")
print(f"q = {q}, B = 3")

# Try a solution
z = np.array([1, 2, -1, 1])
result = (A @ z) % q

print(f"\nTrying z = {z}")
print(f"Az mod {q} = {result}")
print(f"Target b = {b}")
print(f"Match? {np.array_equal(result, b)}")

# Check if short
is_short = np.all(np.abs(z) <= 3)
print(f"z ∈ [-3,3]^m? {is_short}")

if np.array_equal(result, b) and is_short:
    print("\n✓ Valid ISIS solution!")
else:
    print("\n✗ Not a valid solution, keep searching...")

## SIS ≤ ISIS (Reducing SIS to ISIS)

Here's the clever trick to reduce SIS to ISIS:

Given: SIS instance with matrix A ∈ Z_q^(n×m)
Goal: Find z ≠ 0 with Az = 0 (mod q), z short

**Reduction**:
1. Split A = [A' | -b'] where A' is n×(m-1) and b' is n×1
2. Solve ISIS on (A', b'): find z' with A'z' = b' (mod q)
3. Set z = [z', 1]^T
4. Then Az = A'z' - b' = b' - b' = 0 (mod q)
5. And z is short if z' is short!

This shows: if we can solve ISIS, we can solve SIS.

In [ ]:
# Demonstration: SIS to ISIS reduction
q = 13
# SIS instance: 3x5 matrix
A_sis = np.array([
    [1, 0, 7, 12, 4],
    [2, 11, 3, 6, 12],
    [9, 8, 10, 5, 1]
])

print("SIS to ISIS Reduction")
print("=" * 70)
print("Original SIS problem: find z ≠ 0 with Az = 0 (mod q)\n")
print("Matrix A (3×5):")
print(A_sis)

# Split: A = [A' | -b']
A_prime = A_sis[:, :-1]  # First 4 columns
b_prime = (-A_sis[:, -1]) % q  # Last column negated

print("\nSplit A = [A' | -b']:")
print(f"A' (3×4):")
print(A_prime)
print(f"b' (3×1): {b_prime}")

# Suppose we solve ISIS: A'z' = b' (mod q)
# From earlier, we know z = (3, 1, -1, 0, 1) is an SIS solution
# So z' should be the first 4 components
z_sis_solution = np.array([3, 1, -1, 0, 1])
z_prime = z_sis_solution[:-1]
last_entry = z_sis_solution[-1]

print(f"\nISIS solution z': {z_prime}")
print(f"Check A'z' mod {q}: {(A_prime @ z_prime) % q}")
print(f"Target b': {b_prime}")
print(f"Match? {np.array_equal((A_prime @ z_prime) % q, b_prime)}")

# Construct SIS solution
z_constructed = np.append(z_prime, last_entry)
print(f"\nConstruct SIS solution: z = [z', {last_entry}] = {z_constructed}")
print(f"Verify Az mod {q}: {(A_sis @ z_constructed) % q}")
print(f"Is zero? {np.all((A_sis @ z_constructed) % q == 0)}")
print("\n✓ SIS problem solved via ISIS!")

## ISIS ≤ SIS (Reducing ISIS to SIS)

The reverse direction is trickier but also works:

Given: ISIS instance (A, b) where A ∈ Z_q^(n×m)
Goal: Find short z with Az = b (mod q)

**Reduction** (randomized):
1. Pick random position j ∈ [1, m+1] and random nonzero c ∈ [-B, B]
2. Insert -c^(-1)b (mod q) as a new j-th column in A to get A' (now n×(m+1))
3. Solve SIS on A': find short z' ≠ 0 with A'z' = 0 (mod q)
4. If z'[j] = c, extract ISIS solution by removing j-th entry from z'
5. Otherwise try again with different j, c

This shows: if we can solve SIS, we can solve ISIS (with some probability).

The two problems are essentially equivalent!

In [ ]:
# ISIS to SIS reduction demonstration
q = 13
A_isis = np.array([
    [1, 0, 7, 12],
    [2, 11, 3, 6],
    [9, 8, 10, 5]
])
b_isis = np.array([4, 12, 1])

print("ISIS to SIS Reduction")
print("=" * 70)
print("ISIS problem: find short z with Az = b (mod q)")
print(f"\nA (3×4):")
print(A_isis)
print(f"b: {b_isis}")

# Choose position j=4 (last column) and c=1
j = 4  # 1-indexed
c = 1
c_inv = modinv(c, q)
new_col = (-c_inv * b_isis) % q

print(f"\nReduction:")
print(f"  Insert -c^(-1)b as column {j}")
print(f"  c = {c}, c^(-1) mod {q} = {c_inv}")
print(f"  New column: {new_col}")

# Create A' by inserting new column at position j
A_prime_sis = np.column_stack([A_isis, new_col])

print(f"\nA' (3×5) for SIS problem:")
print(A_prime_sis)

# We know z = (3,1,-1,0,1) is an SIS solution for this matrix
z_sis = np.array([3, 1, -1, 0, 1])
print(f"\nSIS solution z': {z_sis}")
print(f"Verify A'z' mod {q}: {(A_prime_sis @ z_sis) % q}")
print(f"Is zero? {np.all((A_prime_sis @ z_sis) % q == 0)}")

# Check if z'[j] = c
if z_sis[j-1] == c:
    print(f"\n✓ Lucky! z'[{j}] = {z_sis[j-1]} = c = {c}")
    # Extract ISIS solution by removing j-th entry
    z_isis = np.delete(z_sis, j-1)
    print(f"Extract ISIS solution: {z_isis}")
    print(f"Verify Az mod {q}: {(A_isis @ z_isis) % q}")
    print(f"Target b: {b_isis}")
    print(f"Match? {np.array_equal((A_isis @ z_isis) % q, b_isis)}")
    print("\n✓ ISIS problem solved via SIS!")
else:
    print(f"\n✗ Unlucky: z'[{j}] = {z_sis[j-1]} ≠ c = {c}")
    print("Would need to try different (j, c) pair")

## Normal Form ISIS

There's a cleaner version of ISIS called normal-form ISIS (nf-ISIS):

**nf-ISIS(n, m, q, B)**: Given A ∈ Z_q^(n×m) and b ∈ Z_q^n, find z ∈ Z^(m+n) such that:
- [A | I_n]z = b (mod q)
- z ∈ [-B, B]^(m+n)

Here I_n is the n×n identity matrix appended to A.

The nice thing about this form is that it's equivalent to ISIS but the structure is cleaner - the identity matrix gives you more flexibility in finding solutions.

**Equivalence**:
- nf-ISIS → ISIS: Multiply by random invertible matrix C
- ISIS → nf-ISIS: Split last n columns and normalize

This form is often used in cryptographic constructions because of its cleaner structure.

In [ ]:
# Normal-form ISIS example
q = 13
n, m = 2, 3

# Random A
A = np.array([
    [5, 7, 3],
    [2, 8, 4]
])

# Target
b = np.array([6, 11])

# Build [A | I_n]
I_n = np.eye(n, dtype=int)
A_extended = np.column_stack([A, I_n])

print("Normal-form ISIS")
print("=" * 70)
print(f"Parameters: n={n}, m={m}, q={q}, B=3")
print(f"\nMatrix A (2×3):")
print(A)
print(f"\n[A | I_2] (2×5):")
print(A_extended)
print(f"\nTarget b: {b}")

# Find solution z ∈ Z^(m+n) = Z^5
# Try z = [1, 0, -1, 2, 3] (first 3 from A side, last 2 from identity side)
z = np.array([1, 0, -1, 2, 3])

result = (A_extended @ z) % q
print(f"\nTrying z = {z}")
print(f"  [A|I]z mod {q}: {result}")
print(f"  Target b: {b}")
print(f"  Match? {np.array_equal(result, b)}")

# Check if short
is_short = np.all(np.abs(z) <= 3)
print(f"  Short (in [-3,3]^5)? {is_short}")

if np.array_equal(result, b) and is_short:
    print("\n✓ Valid nf-ISIS solution!")
    print(f"\nNote: The identity block makes it easier to hit target b")
    print(f"Last n={n} entries of z directly contribute to b")
else:
    print("\n✗ Not valid, adjusting...")

## Summary

**SIS (Short Integer Solutions)**: Find short nonzero z with Az = 0 (mod q)
- Parameters: n (rows), m (columns), q (modulus), B (bound)
- Requirement: m > n and (B+1)^m > q^n for solutions to exist
- Hardness: Believed to be hard based on lattice problems (SVP, CVP)

**Applications**:
- Collision-resistant hash functions: H(z) = Az mod q
- Digital signatures (using ISIS variant)
- Zero-knowledge proofs

**ISIS (Inhomogeneous SIS)**: Find short z with Az = b (mod q)
- Used when you need to hit a specific target b
- Equivalent to SIS (can reduce either way)

**Normal-form ISIS**: ISIS with matrix [A | I_n] instead of just A
- Cleaner structure, same hardness
- Often used in crypto constructions

**Key Insight**: The security of these problems relies on the hardness of finding short vectors in lattices. If you can solve SIS/ISIS efficiently, you can break many lattice-based crypto schemes!

Next up: Ring-SIS and Module-SIS for more efficient constructions.